# Color-based Tracking
moves away from tracking individual pixel gradients (like KLT) and instead focuses on tracking the **statistical distribution of colors** within a region. The two primary algorithms for this are **Mean Shift** and its improved version, **CamShift**.

---

### 1. Mean Shift Tracking

Mean Shift is an iterative method for finding the "peak" or "mode" of a data distribution. In tracking, we use it to find the area in the current frame that most closely matches the **color histogram** of the object from the previous frame.

#### How it works:

1. **Define the Target:** You select a region (ROI) in the first frame.
2. **Color Histogram:** You create a histogram of the target, typically in the **HSV color space** (Hue-Saturation-Value), because it is more robust to lighting changes than RGB.
3. **Backprojection:** In the next frame, every pixel is replaced with its probability of belonging to that color histogram. This creates a "probability map" where the target looks like a bright blob.
4. **The "Shift":** The algorithm starts a window at the last known position and calculates the **center of mass** of the probability map inside that window. It then moves (shifts) the window to that center.
5. **Convergence:** This repeats until the window stops moving—it has found the "peak" concentration of the target color.

---

### 2. CamShift (Continuously Adaptive Mean Shift)

The standard Mean Shift has one major problem: the size of the tracking window stays constant. If an object moves toward the camera (getting larger) or away (getting smaller), Mean Shift fails to adapt.

**CamShift** solves this by:

1. Running Mean Shift first to find the center.
2. **Updating the window size:** It adjusts the window size based on the distribution of the color pixels.
3. **Orientation:** It calculates the orientation of the object (e.g., if a person leans over), allowing the bounding box to rotate.

---

### Comparison: KLT vs. Mean Shift

| Feature | KLT (Gradient-Based) | Mean Shift (Color-Based) |
| --- | --- | --- |
| **What is tracked?** | Specific corner points. | A distribution of colors. |
| **Robustness** | High for rigid objects. | High for non-rigid objects (e.g., a shirt). |
| **Failure Mode** | Fast motion or motion blur. | Similar colored backgrounds. |
| **Input Requirement** | High-contrast textures. | Distinctive color patterns. |


In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

# 1. Capture the initial frame and set the tracking window
ret, frame = cap.read()
# Define initial location of window: (x, y, width, height)
x, y, w, h = 300, 200, 100, 100 
track_window = (x, y, w, h)

# 2. Set up the ROI for tracking
roi = frame[y:y+h, x:x+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Create a mask to ignore low light/low saturation pixels
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria: 10 iterations or move by at least 1pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if not ret: break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 3. Calculate Backprojection (Probability Map)
    dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

    # 4. Apply Mean Shift to get the new location
    ret, track_window = cv2.meanShift(dst, track_window, term_crit)

    # Draw it on image
    x, y, w, h = track_window
    img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), 255, 2)
    
    cv2.imshow('MeanShift Tracking', img2)
    if cv2.waitKey(30) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()